Reverse words
=============

Based on translate.py and on code from xxxxx.

To do
-----

- clean old code.
- do predict.


In [77]:
"""Binary for training translation models and decoding from them.

See the following papers for more information on neural translation models.
 * http://arxiv.org/abs/1409.3215
 * http://arxiv.org/abs/1409.0473
 * http://arxiv.org/abs/1412.2007
"""

import math
import os
import random
import sys
import time

import numpy as np
import tensorflow as tf

from tensorflow.models.rnn.translate import data_utils
from tensorflow.models.rnn.translate import seq2seq_model


learning_rate = 0.5                  # "Learning rate."
learning_rate_decay_factor = 0.9     # "Learning rate decays by this much."
max_gradient_norm = 5.0              # "Clip gradients to this norm."
num_nodes = 128                      # "Size of each model layer." # was 1024 
num_layers = 1                       # "Number of layers in the model." # was 3

steps_per_eval = 10                  # "How many training steps to do per eval."
evals_per_validate = 10
dodecode = False                     # "Set to True for interactive decoding."


# We use a number of buckets and pad to the closest one for efficiency.
# See seq2seq_model.Seq2SeqModel for details of how they work.
# _buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]
_buckets = [(10, 11)]

batch_size = 16


In [67]:
import string

vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z]
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    else:
        print('Unexpected character: %s' % char)
        return 0
  
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character:  
Unexpected character: ï
1 26 0 0
a z  


In [68]:
def get_batch(size):
    encoder_inputs = [np.random.randint(1, vocabulary_size, size) for _ in xrange(batch_size)]
    decoder_inputs = [np.zeros(size+1, dtype = np.int32) for _ in xrange(batch_size)]
    weights = [np.ones(size+1, dtype = np.float32) for _ in xrange(batch_size)]
    for i in xrange(batch_size):
        r = random.randint(1, size-1)
        encoder_inputs[i][r:] = 0
        # Reverse the encoder input sequence, but leave a 0 at index 0 and at least one 0 at the end.
        # These are the GO and EOS markers.
        decoder_inputs[i][1:r+1] = encoder_inputs[i][:r][::-1]
        weights[i][r+1:] = 0.0
    return np.transpose(encoder_inputs), np.transpose(decoder_inputs), np.transpose(weights)


Bacthes are three matrices, encoder_inputs and decoder_inputs and weights. Weights are all zeroes.

encoder_inputs have one entry per column (a fake word) with one letter per row.

decoder_inputs (the "labels") are encoder_inputs reversed.

In [69]:
# batch_size=20
b = get_batch(10)
print('e_i')
print(b[0])
print('d_i')
print(b[1])



e_i
[[21 18 18  3 10  2 22 18 14  5  5 23  6  4  7 16]
 [10  0 22 24 12  9 10 23 19  9  1 18 16  4 12  0]
 [25  0 12 10 17 25  0  1  0  4 20  7 16  6 24  0]
 [12  0 23  5  0 10  0 10  0  0 16 15  4  0 24  0]
 [25  0 13 16  0  9  0 23  0  0 19  5  6  0 24  0]
 [13  0 22 19  0 12  0  4  0  0  0 13  0  0  1  0]
 [ 0  0 20 25  0  0  0 10  0  0  0  0  0  0  8  0]
 [ 0  0  9  3  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0 23 14  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
d_i
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [13 18 23 14 17 12 10 10 19  4 19 13  6  6  1 16]
 [25  0  9  3 12  9 22  4 14  9 16  5  4  4  8  0]
 [12  0 20 25 10 10  0 23  0  5 20 15 16  4  1  0]
 [25  0 22 19  0 25  0 10  0  0  1  7 16  0 24  0]
 [10  0 13 16  0  9  0  1  0  0  5 18  6  0 24  0]
 [21  0 23  5  0  2  0 23  0  0  0 23  0  0 24  0]
 [ 0  0 12 10  0  0  0 18  0  0  0  0  0  0 12  0]
 [ 0  0 22 24  0  0  0  0  0  0  0  0  0  0  7  0]
 [ 0  0 18  3  0  0  0

In [70]:

def get_validation_batch(text, size):
    words = text.split()[:batch_size]
    encoder_inputs = [np.zeros(size, dtype = np.int32) for _ in xrange(batch_size)]
    for i, word in enumerate(words):
        l = min(len(word), size - 1)
        for j in xrange(l):
            encoder_inputs[i][j] = char2id(word[j])
    return np.transpose(encoder_inputs)

def totext(inputs):
    inp = np.transpose(inputs)
    toletters = np.vectorize(id2char)
    return ' '.join([ ''.join(row).rstrip() for row in toletters(inp)])


val_encoder_inputs = get_validation_batch("the quick brown fox jumps over the lazy dog", 10)
print(val_encoder_inputs)
print(totext(val_encoder_inputs))

[[20 17  2  6 10 15 20 12  4  0  0  0  0  0  0  0]
 [ 8 21 18 15 21 22  8  1 15  0  0  0  0  0  0  0]
 [ 5  9 15 24 13  5  5 26  7  0  0  0  0  0  0  0]
 [ 0  3 23  0 16 18  0 25  0  0  0  0  0  0  0  0]
 [ 0 11 14  0 19  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
the quick brown fox jumps over the lazy dog       


----------------------

Model
-----



In [71]:

def create_model(session, forward_only):
    """Create translation model and initialize or load parameters in session."""
    model = seq2seq_model.Seq2SeqModel(
        vocabulary_size, vocabulary_size, _buckets,
        num_nodes, num_layers, max_gradient_norm, batch_size,
        learning_rate, learning_rate_decay_factor,
        use_lstm = True, forward_only=forward_only)
    
    print("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
    return model


In [72]:
# should study this.

def evaluate_model(model, sess, val_encoder_inputs, output_size):
    val_decoder_inputs = np.zeros((output_size+1, batch_size,), dtype = np.int32)
    val_target_weights = np.zeros((output_size+1, batch_size,), dtype = np.float32)
    val_target_weights[0,:] = 1.0
    is_finished = np.full((batch_size,), False, dtype = np.bool_)
    for i in xrange(output_size):
        _, _, output_logits = model.step(sess,
                                         val_encoder_inputs, val_decoder_inputs, val_target_weights,
                                         bucket_id = 0, forward_only = True)
        p = np.argmax(output_logits[i], axis = 1)
        is_finished = np.logical_or(is_finished, p == 0)
        val_decoder_inputs[i,:] = (1 - is_finished) * p
        val_target_weights[i,:] = (1.0 - is_finished) * 1.0
    return val_decoder_inputs

In [80]:
num_steps = 501


def train():

    with tf.Session() as sess:
        # Create model.
        print("Creating %d layers of %d units." % (num_layers, num_nodes))
        model = create_model(sess, False)

        # This is the training loop.
        loss = 0.0
        current_step = 0
        previous_losses = []
        for _ in xrange(num_steps):

            # Get a batch and make a step.
            encoder_inputs, decoder_inputs, target_weights = get_batch(10)
            _, step_loss, _ = model.step(sess,
                                         encoder_inputs, decoder_inputs, target_weights,
                                         0, False)
            loss += step_loss / steps_per_eval
            current_step += 1

            # Once in a while, print statistics, and run evals.
            if current_step % steps_per_eval == 0:
                # Print statistics for the previous epoch.
                perplexity = math.exp(loss) if loss < 300 else float('inf')
                print ("step %d learning rate %.4f perplexity %.2f" %
                        (model.global_step.eval(), model.learning_rate.eval(), perplexity))
                
                # Decrease learning rate if no improvement was seen over last 3 times.
                if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
                    sess.run(model.learning_rate_decay_op)
                previous_losses.append(loss)
                # Zero loss.
                loss = 0.0
                
                # test model.
                if current_step % (steps_per_eval * evals_per_validate) == 0:
                    val_decoder_inputs = evaluate_model(model, sess, val_encoder_inputs, 10)
                    print(totext(val_decoder_inputs))
                
                sys.stdout.flush()


In [81]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

train()


Creating 1 layers of 128 units.
Created model with fresh parameters.
step 10 learning rate 0.5000 perplexity 22.90
step 20 learning rate 0.5000 perplexity 18.16
step 30 learning rate 0.5000 perplexity 15.30
step 40 learning rate 0.5000 perplexity 15.38
step 50 learning rate 0.5000 perplexity 11.10
step 60 learning rate 0.5000 perplexity 10.80
step 70 learning rate 0.5000 perplexity 9.55
step 80 learning rate 0.5000 perplexity 7.92
step 90 learning rate 0.5000 perplexity 6.07
step 100 learning rate 0.5000 perplexity 6.55
ee kci nnbb xf pp re ee yy gg       
step 110 learning rate 0.5000 perplexity 5.75
step 120 learning rate 0.5000 perplexity 5.16
step 130 learning rate 0.5000 perplexity 5.16
step 140 learning rate 0.5000 perplexity 5.12
step 150 learning rate 0.5000 perplexity 4.31
step 160 learning rate 0.5000 perplexity 3.63
step 170 learning rate 0.5000 perplexity 4.24
step 180 learning rate 0.5000 perplexity 3.68
step 190 learning rate 0.5000 perplexity 3.58
step 200 learning rate 

In [5]:
def decode():
  with tf.Session() as sess:
    # Create model and load parameters.
    model = create_model(sess, True)
    model.batch_size = 1  # We decode one sentence at a time.

    # Load vocabularies.
    en_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.en" % FLAGS.en_vocab_size)
    fr_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.fr" % FLAGS.fr_vocab_size)
    en_vocab, _ = data_utils.initialize_vocabulary(en_vocab_path)
    _, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)

    # Decode from standard input.
    sys.stdout.write("> ")
    sys.stdout.flush()
    sentence = sys.stdin.readline()
    while sentence:
      # Get token-ids for the input sentence.
      token_ids = data_utils.sentence_to_token_ids(tf.compat.as_bytes(sentence), en_vocab)
      # Which bucket does it belong to?
      bucket_id = min([b for b in xrange(len(_buckets))
                       if _buckets[b][0] > len(token_ids)])
      # Get a 1-element batch to feed the sentence to the model.
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          {bucket_id: [(token_ids, [])]}, bucket_id)
      # Get output logits for the sentence.
      _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
      # This is a greedy decoder - outputs are just argmaxes of output_logits.
      outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
      # If there is an EOS symbol in outputs, cut them at that point.
      if data_utils.EOS_ID in outputs:
        outputs = outputs[:outputs.index(data_utils.EOS_ID)]
      # Print out French sentence corresponding to outputs.
      print(" ".join([tf.compat.as_str(rev_fr_vocab[output]) for output in outputs]))
      print("> ", end="")
      sys.stdout.flush()
      sentence = sys.stdin.readline()



In [16]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

def self_test():
    """Test the translation model."""
    
    with tf.Session() as sess:
        print("Self-test for neural translation model.")
        # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
        model = seq2seq_model.Seq2SeqModel(10, 10, [(3, 3), (6, 6)], 32, 2,
                                       5.0, 32, 0.3, 0.99, num_samples=8)
        sess.run(tf.initialize_all_variables())

        # Fake data set for both the (3, 3) and (6, 6) bucket.
        data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                    [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
        for _ in xrange(5):  # Train the fake model for 5 steps.
            bucket_id = random.choice([0, 1])
            encoder_inputs, decoder_inputs, target_weights = model.get_batch(data_set, bucket_id)
            model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)

self_test()

Self-test for neural translation model.
